In [439]:
# config
exampleName="IBM_B3"
exampleDescription="Example ontology for IBM Building 3, Dublin"
testing=False

In [440]:
#!pip install rdflib
#!pip install pydot2
#!pip install pydotplus

In [441]:
import rdflib
from rdflib.namespace import RDFS
from rdflib import URIRef, BNode, Literal
import re
from collections import defaultdict
import numpy as np
import pandas as pd
#import pydot as pydot
import networkx as nx
from networkx.drawing.nx_pydot import write_dot
#import matplotlib.pyplot as plt

In [442]:
def ns(url):
  url = url.replace("http://buildsys.org/ontologies/Brick#", "")
  url = url.replace("http://buildsys.org/ontologies/BrickFrame#", "")
  url = url.replace("http://buildsys.org/ontologies/BrickTag#", "")
  return url

BRICKF = rdflib.Namespace('http://buildsys.org/ontologies/BrickFrame#')
TAGS   = rdflib.Namespace('http://buildsys.org/ontologies/BrickTag#')
TS     = rdflib.Namespace('http://buildsys.org/ontologies/Brick#')

g = rdflib.Graph()
g.bind( 'bf', BRICKF)
g.bind('tag', TAGS)
g.bind( 'ts', TS)
result = g.parse('../BuildingSchema/BrickFrame.ttl', format='n3')
result = g.parse('../BuildingSchema/BrickTag.ttl', format='n3')
result = g.parse('../BuildingSchema/Brick.ttl', format='n3')
print(len(g))

13719


define your own namespace

In [443]:
MyNS = rdflib.Namespace('http://buildsys.org/ontologies/examples/'+exampleName+'#')
g.bind(':', MyNS)

### Load TagSets

In [444]:
qres = g.query("""SELECT DISTINCT ?ts WHERE {  ?ts rdfs:subClassOf+ bf:Tag . }""")
brickTags=set()
for row in qres:
  ts=ns(row['ts'])
  brickTags.add(ns(row['ts']))
len(brickTags)

271

In [445]:
qres = g.query("""SELECT DISTINCT ?ts WHERE {  ?ts rdfs:subClassOf+ bf:TagSet . }""")
brickTagSets=set()
brickTagSetTags={}
for row in qres:
  ts=ns(row['ts'])
  brickTagSets.add(ts)
  brickTagSetTags[ts]=set(ts.split('_'))
len(brickTagSets)

758

In [446]:
qres = g.query("""SELECT DISTINCT ?ts WHERE {  ?ts rdfs:subClassOf+ ts:Location . }""")
loc_tags=set(['Location'])
for row in qres:
  ts=ns(row['ts'])
  loc_tags.add(ts)
len(loc_tags)

6

In [447]:
qres = g.query("""SELECT DISTINCT ?ts WHERE {  ?ts rdfs:subClassOf+ ts:Point . }""")
point_tags=set(['Point'])
pointTagSetTags={}
for row in qres:
  ts=ns(row['ts'])
  point_tags.add(ts)
  pointTagSetTags[ts]=set(ts.split('_'))
len(point_tags)

583

In [448]:
qres = g.query("""SELECT DISTINCT ?ts WHERE {  ?ts rdfs:subClassOf+ ts:MeasurementProperty . }""")
measurment_tags=set(['MeasurementProperty'])
for row in qres:
  ts=ns(row['ts'])
  measurment_tags.add(ns(row['ts']))
len(measurment_tags)

100

In [449]:
qres = g.query("""SELECT DISTINCT ?ts WHERE {  ?ts rdfs:subClassOf+ ts:Equipment . }""")
asset_tags=set(['Asset'])
for row in qres:
  ts=ns(row['ts'])
  asset_tags.add(ns(row['ts']))
len(asset_tags)

50

remove hierachical tags

In [450]:
qres = g.query("""SELECT DISTINCT ?ts WHERE {  ?ts bf:isHierarchical "". }""")
for row in qres:
  ts=ns(row['ts'])
  if ts in brickTags: brickTags.remove(ts)
  if ts in brickTagSets: brickTagSets.remove(ts)
  if ts in brickTagSetTags: del brickTagSetTags[ts]
  if ts in point_tags: point_tags.remove(ts)
  if ts in asset_tags: asset_tags.remove(ts)
  if ts in measurment_tags: measurment_tags.remove(ts)
  if ts in pointTagSetTags: del pointTagSetTags[ts]
len(brickTagSets)
len(point_tags)

482

In [451]:
def IndivName(name):
  return re.sub(r'\s','_',re.sub(r'[^\d\w\s]', '', name))

### Load Data

In [452]:
df=pd.read_csv('IBM_B3_points.csv')
if testing: df=df.head(20)
df.head()

,Label,TagSet,AssetType,Asset,AssetParent,City,Building,Wing,Floor,Room,Zone
0,1F_MID_OPENOFF_CO2,Zone_Air_CO2_Sensor,NaN,NaN,NaN,Dublin,B3,SOR42,FirstFloor,OpenOffice,Middle
1,1F_NRTH_OPENOFF_CO2,Zone_Air_CO2_Sensor,NaN,NaN,NaN,Dublin,B3,SOR42,FirstFloor,OpenOffice,North
2,1F_STH_OPENOFF_CO2,Zone_Air_CO2_Sensor,NaN,NaN,NaN,Dublin,B3,SOR42,FirstFloor,OpenOffice,South
3,421_U10_CLG_VLV,FCU_Cooling_Valve_Percentage_Command,FCU,U10,NaN,Dublin,B3,SOR42,FirstFloor,SOR42_1_U10,NaN
4,421_U10_DAT,FCU_Supply_Air_Temperature_Sensor,FCU,U10,NaN,Dublin,B3,SOR42,FirstFloor,SOR42_1_U10,NaN


Analyze Dataset

In [453]:
dfTags=set()
dfTagSets=set(pd.unique(df.TagSet.dropna().ravel()))
for ts in dfTagSets:
  dfTags.update(ts.split('_'))
len(dfTags)

91

In [454]:
dfMissingTags=dfTags - brickTags # schemaUsedTags - 
print("Missing Tags from Bricks:" + str(len(dfMissingTags)))
print(dfMissingTags)

Missing Tags from Bricks:31
{'Circualation', 'Lux', 'Instantations', 'Wireless', 'Incoming', 'External', 'ThermalWheel', 'Accumulated', 'Line', 'In', 'Illumination', 'Elevation', 'Loop', 'Electric', 'Weather', 'Sun', 'Socket', 'Presence', 'Radiation', 'PowerUnit', 'Facade', 'Out', 'Sunblind', 'SRVC', 'Light', 'Footfall', 'Phase', 'Percentage', 'Extract', 'Factor', 'Blind'}


Find best matching tag sets for the defined ones

In [455]:
closestMatches={}
fullMatches={}
for ts in pd.unique(df.TagSet.dropna().ravel()):
  if ts in pointTagSetTags:
    closestMatches[ts]=[ts]
    fullMatches[ts]=ts
    print("Match: "+ts)
  else:
    tss=set(ts.split('_'))
    largestSubset=[]
    largestIntersect=0
    for ts2 in pointTagSetTags:
      inters=tss.intersection(pointTagSetTags[ts2])
      if len(inters)>largestIntersect:
        largestIntersect=len(inters)
        largestSubset=[] # reset largestSubset
      if len(inters)==largestIntersect:
        largestSubset.append(ts2) # add to the largest sets
    smalestSubset=largestSubset
    if len(largestSubset)>1:
      smalestDiff=len(tss)
      smalestSubset=[]
      for ts2 in largestSubset:
        diffs=pointTagSetTags[ts2] - tss
        if len(diffs)<smalestDiff:
          smalestDiff=len(diffs)
          smalestSubset=[] # reset largestSubset
        if len(diffs)==smalestDiff:
          smalestSubset.append(ts2) # add to the largest sets
    closestMatches[ts]=smalestSubset
    print("Best "+ts+":"+str(smalestSubset))

Best Zone_Air_CO2_Sensor:['Outside_Air_CO2_Sensor', 'Return_Air_CO2_Sensor']
Best FCU_Cooling_Valve_Percentage_Command:['Cooling_Valve_Command']
Best FCU_Supply_Air_Temperature_Sensor:['Supply_Air_Temperature_Sensor']
Best FCU_Fan_Speed_Sensor_Command:['Return_Fan_Differential_Speed_Sensor']
Best FCU_Fault_Sensor:['Luminance_Sensor', 'CO2_Sensor', 'Occupancy_Sensor', 'Energy_Sensor', 'Rain_Sensor', 'Humidity_Sensor', 'Fault_Status', 'Hail_Sensor', 'Frost_Sensor']
Best FCU_Heating_Valve_Percentage_Command:['Heating_Valve_Command']
Best FCU_Return_Air_Temperature_Sensor:['Return_Air_Temperature_Sensor']
Best FCU_Air_Temperature_Setpoint:['Discharge_Air_Temperature_Setpoint', 'Mixed_Air_Temperature_Setpoint', 'Supply_Air_Temperature_Setpoint']
Best Light_Load_Electric_Meter:['Power_Meter', 'Water_Meter']
Best Socket_Load_Electric_Meter:['Power_Meter', 'Water_Meter']
Best Electric_Meter_FCU:['Power_Meter', 'Water_Meter']
Best Electric_Meter:['Power_Meter', 'Water_Meter']
Best Electric_Mete

Manual Mapping based on the results.

In [456]:
myMap={
'AHU_Automatic_Mode_Command':'Automatic_Mode_Command',
'AHU_Cooling_Valve_Percentage_Command':'Cooling_Valve_Command',
'AHU_Enable_Command':'AHU_System_Enable_Command',
'AHU_Frost_Open_Sensor':'Frost_Sensor',
'AHU_Heating_Valve_Percentage_Command':'Heating_Valve_Command',
'AHU_Return_Air_Temperature_Sensor':'Return_Air_Temperature_Sensor',
'AHU_Return_Fan_Variable_Speed_Percentage_Command':'Return_Fan_Command',
'AHU_Return_Fan_Variable_Speed_Speed_Sensor':'Return_Fan_Differential_Speed_Sensor',
'AHU_Return_Heat_Valve_Command':'Return_Heat_Valve_Command',
'AHU_Supply_Air_Temperature_Sensor':'Supply_Air_Temperature_Sensor',
'AHU_Supply_Air_Temperature_Setpoint':'Supply_Air_Temperature_Setpoint',
'AHU_Supply_Fan_Flow_Status_Sensor':'Supply_Fan_Status',
'AHU_Supply_Fan_Variable_Speed_Command':'Supply_Fan_Command',
'AHU_Supply_Fan_Variable_Speed_Percentage_Command':'Supply_Fan_Command',
'AHU_Supply_Fan_Variable_Speed_Sensor':'Motor_Speed',
'AHU_Supply_Fan_Variable_Speed_Setpoint':'Return_Supply_Fan_Differential_Speed_Setpoint',
'AHU_ThermalWheel_Effective_Sensor':'Heat_Wheel_Supply_Air_Temperature',
'AHU_Trace_Heat_Sensor':'Trace_Heat_Sensor',
'AHU_Zone_Air_Temperature_Sensor':'Zone_Temperature_Sensor',
'Chilled_Water_Loop_Accumulated_PowerUnit_Meter':'Power_Meter',
'Chilled_Water_Loop_Accumulated_Volume_Meter':'Water_Meter',
'Chilled_Water_Loop_Flow_Meter':'Chilled_Water_Supply_Flow_Sensor',
'Chilled_Water_Loop_Return_Temperature_Sensor':'Chilled_Water_Return_Temperature_Sensor',
'Chilled_Water_Loop_Supply_Temperature_Sensor':'Chilled_Water_Supply_Temperature_Sensor',
'Cold_Water_Storage_Booster_Pump_Phase_Meter':'Power_Meter',
'Electric_Meter':'Power_Meter',
'Electric_Meter_AHU_Phase':'Power_Meter',
'Electric_Meter_FCU':'Power_Meter',
'Electric_Meter_Light':'Power_Meter',
'Electric_Meter_SRVC':'Power_Meter',
'External_Fan_Status_Sensor':'Exhaust_Fan_Status',
'External_Fan_Variable_Speed_Command':'Speed_Command',
'Extract_Fan_Phase_Meter':'Power_Meter',
'FCU_Air_Temperature_Setpoint':'Supply_Air_Temperature_Setpoint',
'FCU_Cooling_Valve_Percentage_Command':'Cooling_Valve_Command',
'FCU_Fan_Run_Time_Sensor':'Run_Time_Sensor',
'FCU_Fan_Speed_Sensor_Command':'Speed_Command',
'FCU_Fault_Sensor':'Fault_Status',
'FCU_Frequency_Electric_Meter':'Power_Meter',
'FCU_Frequency_Electric_Meter':'Power_Meter',
'FCU_Heating_Valve_Percentage_Command':'Heating_Valve_Command',
'FCU_Line_Voltage_Electric_Meter':'Power_Meter',
'FCU_Phase_Average_PowerUnit_Electric_Meter':'Power_Meter',
'FCU_Phase_Current_Electric_Meter':'Power_Meter',
'FCU_Phase_Instantations_PowerUnit_Electric_Meter':'Power_Meter',
'FCU_Phase_Max_PowerUnit_Electric_Meter':'Power_Meter',
'FCU_Power_Factor_Electric_Meter':'Power_Meter',
'FCU_Return_Air_Temperature_Sensor':'Return_Air_Temperature_Sensor',
'FCU_Status_Sensor':'System_Status',
'FCU_Supply_Air_Temperature_Sensor':'Supply_Air_Temperature_Sensor',
'Facade_Sunblind_Sensor':'Luminance_Sensor',
'Footfall_In_Sensor':'Occupancy_Sensor',
'Footfall_Out_Sensor':'Occupancy_Sensor',
'Hot_Water_Loop_Accumulated_PowerUnit_Meter':'Power_Meter',
'Hot_Water_Loop_Accumulated_Volume_Meter':'Hot_Water_Meter',
'Hot_Water_Loop_Flow_Meter':'Hot_Water_Meter',
'Hot_Water_Loop_Return_Temperature_Sensor':'Hot_Water_Return_Temperature_Sensor',
'Hot_Water_Loop_Supply_Temperature_Sensor':'Hot_Water_Supply_Temperature_Sensor',
'Incoming_Water_Meter':'Water_Meter',
'Light_Load_Electric_Meter':'Power_Meter',
'Light_Load_Frequency_Electric_Meter':'Power_Meter',
'Light_Load_Line_Voltage_Electric_Meter':'Power_Meter',
'Light_Load_Phase_Average_PowerUnit_Electric_Meter':'Power_Meter',
'Light_Load_Phase_Current_Electric_Meter':'Power_Meter',
'Light_Load_Phase_Instantations_PowerUnit_Electric_Meter':'Power_Meter',
'Light_Load_Phase_Max_PowerUnit_Electric_Meter':'Power_Meter',
'Light_Load_Power_Factor_Electric_Meter':'Power_Meter',
'Outside_Illumination_Lux_Sensor':'Luminance_Sensor',
'Outside_Radiation_Sensor':'Solar_Radiance_Sensor',
'Rain_Water_Circualation_Pump_Phase_Meter':'Power_Meter',
'Rain_Water_Control_Pump_Meter':'Water_Meter',
'Socket_Load_Blind_Electric_Meter':'Power_Meter',
'Socket_Load_Electric_Meter':'Power_Meter',
'Socket_Load_Frequency_Electric_Meter':'Power_Meter',
'Socket_Load_Line_Voltage_Electric_Meter':'Power_Meter',
'Socket_Load_Phase_Average_PowerUnit_Electric_Meter':'Power_Meter',
'Socket_Load_Phase_Current_Electric_Meter':'Power_Meter',
'Socket_Load_Phase_Electric_Meter':'Power_Meter',
'Socket_Load_Phase_Instantations_PowerUnit_Electric_Meter':'Power_Meter',
'Socket_Load_Phase_Max_PowerUnit_Electric_Meter':'Power_Meter',
'Socket_Load_Power_Factor_Electric_Meter':'Power_Meter',
'Socket_Load_Wireless_Electric_Meter':'Power_Meter',
'Solar_Radiation_Sensor':'Solar_Radiance_Sensor',
'Sun_Azimuth_Sensor':'Solar_Azimuth_Angle_Sensor',
'Sun_Elevation_Sensor':'Solar_Azimuth_Angle',
'Switch_Emergency_Command':'Emergency_Power_Off_Switch_Command',
'Water_Meter_Rain':'Water_Meter',
'Weather_Hail_Sensor':'Hail_Sensor',
'Weather_Outside_Air_Temperature_Sensor':'Outside_Air_Temperature_Sensor',
'Weather_Pressure_Sensor':'Static_Pressure_Sensor',
'Weather_Rain_Duration_Sensor':'Rain_Duration_Sensor',
'Weather_Rain_Sensor':'Rain_Sensor',
'Weather_Relative_Humidity_Sensor':'Humidity_Sensor',
'Weather_Wind_Speed_Average_Sensor':'Wind_Speed_Sensor',
'Weather_Wind_Speed_Direction_Sensor':'Wind_Direction_Sensor',
'Weather_Wind_Speed_Max_Sensor':'Wind_Speed_Sensor',
'Zone_Air_CO2_Sensor':'CO2_Sensor',
'Zone_Air_Temperature_Sensor':'Zone_Temperature_Sensor',
'Zone_Illumination_Command':'Luminance_Command',
'Zone_Illumination_Fault_Sensor':'Luminance_Alarm',
'Zone_Illumination_Sensor':'Luminance_Sensor',
'Zone_Presence_Sensor':'Occupancy_Sensor'
}
for ts in myMap: 
  fullMatches[ts]=myMap[ts]
for ts in fullMatches.values():
  dfTags.update(ts.split('_'))

In [457]:
for ts in closestMatches:
  if ts not in fullMatches:
    print(ts+":"+str(closestMatches[ts]))

write header

In [458]:
fo = open(exampleName+'.ttl', 'w')
fo.write("""@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix DUL: <http://www.loa-cnr.it/ontologies/DUL.owl#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix ssn: <http://purl.oclc.org/NET/ssnx/ssn#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix unit: <http://qudt.org/vocab/unit#> .
@prefix gbXML: <http://www.gbxml.org/schema#> .
@prefix haystack: <http://project-haystack.org/tag/> .\n""")
fo.write("@prefix bf:  <http://buildsys.org/ontologies/BrickFrame#> .\n")
fo.write("@prefix tag: <http://buildsys.org/ontologies/BrickTag#> .\n")
fo.write("@prefix ts:  <http://buildsys.org/ontologies/Brick#> .\n")
fo.write("@prefix : <"+str(MyNS)+"> .\n\n")
fo.write("<"+str(MyNS).replace("#","")+">  a owl:Ontology ;\n")
fo.write("\towl:imports <http://buildsys.org/ontologies/Brick> ;\n")
fo.write('\trdfs:comment "'+exampleDescription+'"@en .\n\n')

66

write locations

In [459]:
# create location individuals
locationCols=["City","Building","Wing","Floor","Room","Zone"]
locations={}
for col in locationCols:
  for indiv in pd.unique(df[col].dropna().ravel()):
    if indiv!="":
      indivLocName=IndivName(col+"_"+indiv)
      fo.write("\n :"+indivLocName+"  a   tag:"+col+";")
      fo.write('\n\t\t\t rdfs:label "'+indiv+'"@en .\n')
      locations[indiv]=indivLocName
# add relations
for idx in df.index:
  for ci in range(1,len(locationCols)):
    childIndiv =df.loc[idx,locationCols[ci]]
    parentIndiv=df.loc[idx,locationCols[ci-1]]
    if childIndiv in locations and parentIndiv in locations:
      fo.write("\n :"+locations[childIndiv]+"  bf:isPartOf   :"+locations[parentIndiv]+".")
      fo.write("\n :"+locations[parentIndiv]+"  bf:hasPart   :"+locations[childIndiv]+".\n")

write assets

In [460]:
# create asset individuals
assets={}
for idx in df.index:
  assetName = str(df.loc[idx, "Asset"])
  if assetName=="" or assetName=="nan" or assetName in assets: 
    continue
  assetType = df.loc[idx, "AssetType"]
  indivLocName=IndivName(assetType+"_"+assetName)
  fo.write("\n :"+indivLocName+"  a   tag:"+assetType+";")
  fo.write('\n\t\t\t rdfs:label "'+assetName+'"@en .\n')
  assets[assetName]=indivLocName
  assetParent = str(df.loc[idx, "AssetParent"])
  if assetParent!="" and assetParent!="nan":
    if assetParent not in assets:
      parentLocName=IndivName(assetType+"_"+assetParent)
      fo.write("\n :"+parentLocName+"  a   tag:"+assetType+";")
      fo.write('\n\t\t\t rdfs:label "'+assetParent+'"@en .\n')
      assets[assetParent]=parentLocName
    fo.write("\n :"+assets[assetName]+"  bf:isPartOf   :"+assets[assetParent]+".")
    fo.write("\n :"+assets[assetParent]+"  bf:hasPart   :"+assets[assetName]+".\n")

analyze tag sets

In [461]:
#create individual for tags
for tag in dfTags:
  if tag in brickTags:
    fo.write("\n :"+tag+"0  a  tag:"+tag+".\n")
  else:
    fo.write("\n :"+tag+"  rdfs:subClassOf  bf:Tag.")
    fo.write("\n :"+tag+"0  a  :"+tag+".\n")

In [462]:
for idx in df.index:
  pointIndivName = IndivName(df.loc[idx, "Label"])
  pointType = df.loc[idx, "TagSet"]
  if pointType in fullMatches:
    tags=set(pointType.split('_')) | set(fullMatches[pointType].split('_'))
    fo.write("\n :"+pointIndivName+"  a  bf:Label, bf:TagSet, ts:"+fullMatches[pointType]+";")
  else:
    tags=set(pointType.split('_'))
    fo.write("\n :"+pointType+"  rdfs:subClassOf   bf:TagSet.")
    fo.write("\n :"+pointIndivName+"  a   bf:Label, bf:TagSet, :"+pointType+";")
  fo.write('\n \t\t\t rdfs:label "'+df.loc[idx, "Label"]+'"@en ;')
  # write tags
  for tag in tags:
    fo.write("\n \t\t\t bf:hasTag   :"+tag+"0;")
  # write location
  loc=df.loc[idx, locationCols].dropna().ravel()[-1]
  fo.write("\n \t\t\t bf:isLocatedIn :"+locations[loc]+'. \n')
  fo.write("\n :"+locations[loc]+"  bf:hasPoint  :"+pointIndivName+".")
  # write assets
  assetName = str(df.loc[idx, "Asset"])
  if assetName!="" and assetName!="nan":
    fo.write("\n :"+assets[assetName]+"  bf:hasPoint  :"+pointIndivName+".")
    fo.write("\n :"+pointIndivName+"  bf:isPointOf  :"+assets[assetName]+".\n")

In [463]:
fo.close()